In [ ]:
import pandas as pd
from datetime import datetime
from time import mktime
from tqdm.notebook import tqdm
import requests
import json 
from pathlib import Path

In [ ]:
api_url = 'https://api.darksky.net/forecast'
api_key = 'API_KEY'
# Uncomment to stuff files someplace other than the current directory.  This will create it
# data_dir = Path('WHERE/YOU/WANT/THE/DATA').mkdir(parents=True, exist_ok=True)
data_dir = Path.cwd()

URL format:
```
https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]
```
or
```
url = f"{api_url}/{api_key}/{latitude},{longitude},{time}"
```


## Load a DF with the lat/long and dates we want

In [ ]:
df = pd.read_csv('./api_events.csv', index_col=0)

In [ ]:
print(len(df))
df.head()

In [ ]:
# We're gonna use loops, so let's not use pandas anymore, lest we be shamefully mocked
records = df.to_dict(orient='records')

In [ ]:
# In case we don't want to do this all at once:
limit = 500

In [ ]:
# This let's us write to the status bar.  It's cute
pbar = tqdm(records[:limit])
for record in pbar:
    # See?!?
    pbar.set_description(f"[{record['lat']:.5}/{record['long']:.5}] {record['index']}")
    
    # If we're restarting batches and we've already written this file, don't fetch it again
    # (hopefully we didn't write garbage)
    if Path(data_dir / f"{record['index']}.json").is_file():
        continue
       
    # Set our URL params
    lat = record['lat']
    lng = record['long']
    dt = mktime(datetime.strptime(record['date'], '%Y-%m-%d').timetuple()) 
    
    url = f"{api_url}/{api_key}/{lat},{lng},{dt}"
   
    # Test API URL
    # url = f"https://www.metaweather.com/api/location/search/?lattlong={lat},{lng}"
    try:
        resp = requests.get(url)
        resp.raise_for_status()
    except:
        # The only thing worse than nakedly catching all exceptions...
        pass
    
    try:
        doc = resp.json()
    except:
        # ...is silently masking them
        pass
    else:
        with open(Path(data_dir / f"{record['index']}.json"), 'w') as f:
            json.dump(doc, f)
        
    # Should we sleep?